In [2]:
'''
DESCRIPTION 
    Utility function that chops a sequence into several reads with bounded random lengths that 
    have a bounded random overlap
INPUT
    sequence       | a sequence of characters that will be divided into overlapping subsequences
    min_subseq_len | the shortest length a subsequence can have
    max_subseq_len | the longest length a subsequence can have
    min_overlap    | the shortest overlap two subsequences can share
    max_overlap    | the longest overlap two subsequences can share
    circularize    | boolean indicating whether to add a random amount of the end of the sequence
                   | to the beginning and vice versa
    seed           | random seed for the random function for reproducibility
OUTPUT
    A list of overlapping reads of random bounded size which share a bounded random amount of
    overlap
'''
def generate_reads(sequence,min_subseq_len,max_subseq_len,min_overlap,max_overlap,min_coverage=None,circularise=False,seed=None):
    import random

    random.seed(seed)
    if circularise: sequence = sequence[-random.randint(min_overlap,max_overlap):] + sequence + sequence[:random.randint(min_overlap,max_overlap)]
    reads = []
    while 1: 
        start = 0
        end = random.randint(min_subseq_len,max_subseq_len)
        reads += [sequence[start:end]]
        while end < len(sequence):
            start = random.randint(end-max_overlap,end-min_overlap)
            if (len(sequence) - start)/max_subseq_len < 2:
                if (len(sequence) - start)/max_subseq_len < 1:
                    end = len(sequence)
                else:
                    a = 0
                    while (len(sequence) - start)/(min_subseq_len+a) > 2: a+=1
                    end = random.randint(start+min_subseq_len+a,start+max_subseq_len) 
            else: end = random.randint(start+min_subseq_len,start+max_subseq_len) 
            reads += [sequence[start:end]]
        if min_coverage is None or len(set(reads))*(sum(len(read) for read in set(reads))/len(set(reads)))/len(sequence) >= min_coverage: return list(set(reads))

'''
DESCRIPTION 
    Utility function that creates a random sequence containing only the letters A, T, G, and C
INPUT
    n          | the length of the sequence
    palindrome | a boolean indicating whether the sequence must be a palidrome or not
    seed       | random seed for the random function for reproducibility
OUTPUT
    A random sequence of length n
'''
def generate_genome_sequence(n,palindrome=False,seed=None):
    import random,math
    
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    if palindrome: n = math.ceil(n/2)
    for _ in range(n):
        seq += nucleotides[random.randint(1,4)]
    if palindrome: seq += ''.join(reversed(seq[:int(n-math.fmod(n,2))]))
    return seq

# Sequitur

In [3]:
# ! pip install networkx jellyfish
import jellyfish
import networkx as nx
import math
# from networkx import bipartite

In [4]:
def build_suffix_array(reads,min_suf_len=3):
    suf_arr = []
    for read in reads:
        read += '$' + str(reads.index(read))
        for i in range(len(read)):
            if len(read[i:]) < min_suf_len + 2: continue 
            suf_arr += [read[i:]]
    suf_arr.sort()
    suf_arr_ind = []
    for s in range(len(suf_arr)):
        suf_arr_ind += [int(suf_arr[s].split('$')[-1].__str__())]
        suf_arr[s] = suf_arr[s][:suf_arr[s].find('$')+1]
    return suf_arr,suf_arr_ind

def build_bipartite_graph(reads,suf_arr=None,suf_arr_ind=None,max_diff=0.25,min_suf_len=3):
    if suf_arr is None or suf_arr_ind is None: suf_arr,suf_arr_ind = build_suffix_array(reads)
    B = nx.DiGraph()
    for read in reads:
        for j in range(min_suf_len + 1):
            i = suf_arr.index(read[j:]+'$') - 1
            while normalised_damerau_levenshtein_distance(read,suf_arr[i][:-1]) <= 0.5:
                if not reads[suf_arr_ind[i]] == read and \
                   not B.has_edge('^' + reads[suf_arr_ind[i]],read + '$') and \
                   normalised_damerau_levenshtein_distance(read,suf_arr[i][:-1]) < max_diff and \
                   read.startswith(suf_arr[i][:-1]): 
                        B.add_edge('^' + reads[suf_arr_ind[i]],read + '$',weight=len(suf_arr[i][:-1]))
                i -= 1
    return B.to_undirected()

def build_virtual_graph(graph,reads,draw=False):
    from networkx import bipartite

    G_v = nx.DiGraph()
    # for p,s in nx.maximal_matching(graph):
    for p,s in bipartite.maximum_matching(graph,top_nodes=set('^' + read for read in reads).intersection(graph.nodes())).items():
        if s[-1] == '$':
            if s[:-1] in reads: reads.remove(s[:-1])
            G_v.add_edge(p[1:],s[:-1],weight=nx.get_edge_attributes(graph,"weight")[(p,s) if (p,s) in nx.get_edge_attributes(graph,"weight") else (s,p)])
        else: 
            if p[:-1] in reads: reads.remove(p[:-1])
            G_v.add_edge(s[1:],p[:-1],weight=nx.get_edge_attributes(graph,"weight")[(p,s) if (p,s) in nx.get_edge_attributes(graph,"weight") else (s,p)])
    if draw: nx.draw(G_v)
    return G_v,reads

def sequitur(bipartite_graph,virtual_graph,reads,draw=False):
    while not nx.is_connected(virtual_graph.to_undirected()):
        add_edges = []
        remove_edges = []
        for node in reads:
            # if degree == 0:
            a_edge = (None,None,0)
            r_edge = (None,None)
            for in_p,in_s,in_w in bipartite_graph.edges([node+'$','^'+node],data="weight"):
                for out_p,out_s,out_w in virtual_graph.edges(in_s[1:] if in_s.startswith('^') else in_s[:-1],data='weight'):
                    if (out_w < in_w and a_edge[2] < in_w) or a_edge[0] is None: 
                        a_edge = (out_p,node,in_w) if in_s.startswith('^') else (node,out_p,in_w)
                        r_edge = (out_p,out_s)
            if a_edge[0] is not None: 
                add_edges += [(a_edge[0],a_edge[1],{'weight':a_edge[2]})]
                remove_edges += [r_edge]
        virtual_graph.remove_edges_from(remove_edges)
        for _,node in remove_edges:
            if virtual_graph.in_degree(node) == 0 and node not in reads: reads += [node]
        virtual_graph.add_edges_from(add_edges)
        for _,node,_ in add_edges:
            if virtual_graph.in_degree(node) > 0 and node in reads: reads.remove(node)
        if draw: nx.draw(virtual_graph)
    return virtual_graph

def find_path(path):
    while path.number_of_nodes() > 1:
        path.add_node(list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:])
        predecessors = list(path.predecessors(list(path.edges)[0][0]))
        for pre in predecessors:
            path.add_edge(pre,list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:],weight=path.edges[pre,list(path.edges)[0][0]]['weight'])
            path.remove_edge(pre,list(path.edges)[0][0])
        successors = list(path.successors(list(path.edges)[0][1]))
        for suc in successors:
            path.add_edge(list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:],suc,weight=path.edges[list(path.edges)[0][1],suc]['weight'])
            path.remove_edge(list(path.edges)[0][1],suc)
        path.remove_nodes_from(list(path.edges)[0])
    return list(path.nodes())[0]

def normalised_damerau_levenshtein_distance(read,overlap):
    import jellyfish

    return jellyfish.damerau_levenshtein_distance(read.__str__()[:min(len(overlap),len(read))],overlap.__str__()[:min(len(overlap),len(read))])/min(len(overlap),len(read))

In [7]:
sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']

G = build_bipartite_graph(reads)
G_v,zero_in = build_virtual_graph(G,reads.copy())
path = sequitur(G,G_v,zero_in)
find_path(path)

'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'

In [9]:
sequence = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
G = build_bipartite_graph(reads)
G_v,zero_in = build_virtual_graph(G,reads.copy())
path = sequitur(G,G_v,zero_in)
find_path(path)

'you say hello world, i bellow go to hell'

In [11]:
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
G = build_bipartite_graph(reads)
G_v,zero_in = build_virtual_graph(G,reads.copy())
path = sequitur(G,G_v,zero_in)
find_path(path)

'she_sells_sea_shells_on_the_sea_shore'

In [12]:
seed = 0
sequence = generate_genome_sequence(10000,seed=seed)
reads = generate_reads(sequence,250,500,50,100,seed=seed)
G = build_bipartite_graph(reads)
G_v,zero_in = build_virtual_graph(G,reads.copy())
path = sequitur(G,G_v,zero_in)
find_path(path)

'TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATGGTATCACATTGACAAACACGGCATTAAGTAGCGACGAAACGGGATTTGCCTGACCGGGGAGAAGCCGGTCGATCAGCAGTGGTAATTGGATATTAGGCCTAAACCATAATGTTCTAGCGCTCGAAATCATTGCACCACTTGCATCTTTGTTCCAGGGACGCTGTAAAACCAGATGCCTGTAAATCGTTTCAACGGGATGGTTTACCCGGAATTCTACGTATTTAATCAACGAGCTTAATGAGCTGACATTGCTGAAATGACCATGACTTAATAATCATTTATGGAGAAGAGGCACGACCACAAGGACCCTATGGCACGGTGGGCAAGCTCCCGCCCGGTACATAACTGTCTGGACTGATTATGTCGGTACAGACTTCTTCCTGCGTATCGATTACGAGCTTATCTGAAGAAGTTTAGGGCAAAGGGACCATGGCCATTGGTGCCAATTTCGGTTCTTGTATGCTACAGTTAAATAGAAAGGCCGCATTGTCGTTCTCGCCCTGTTTTCCTCATACACGACCGAGGTTATTTGTCGGAAACGAGACATCTCTCGAAGGTGGAACGACGCCGGGTGTGCAGAATTTATTTTAAACACTCTATTACCTCCGGGTAGCGTTGGCAAACTCCGATAATGAGCGCCAGGCGTGCCAGGACTCCACCTCCCCTGCTAAGTTGACCTTGAGCTCGGTACAGCGTCGGCGAGACGATAACAACGAAGTCCTTCGGCGTTATGTAATTCACCAGCCCACCATATCAGGTAATAGGCTCGCTGGTTAGGTAGATTATGTAAGAGGCGTGCAG

In [13]:
successes = 0
n = 10
for seed in range(n):   
    sequence = generate_genome_sequence(10000,seed=seed)
    reads = generate_reads(sequence,250,500,50,100,seed=seed)
    G = build_bipartite_graph(reads)
    G_v,zero_in = build_virtual_graph(G,reads.copy())
    path = sequitur(G,G_v,zero_in)
    
    
    s = '| Seed: ' + str(seed) + ' | '
    if find_path(path) == sequence:
        s+='SUC | ' + str(find_path(path)) + ' == ' + sequence
        successes+=1
    # elif type(sequitur.sequence) is list and all(s in sequence for s in sequitur.sequence):
    #     s+='PAR | ' + str(sequitur.sequence) + ' ~~ ' + sequence
    #     successes+=0.5
    else: s+='FAI | ' + find_path(path) + ' != ' + sequence
    print(s)
    print('-----------------------------------------')
print('ACCURACY: '+str((successes/n)*100)+'%')

| Seed: 0 | SUC | ...
-----------------------------------------
| Seed: 1 | SUC | ...
-----------------------------------------
| Seed: 2 | SUC | ...
-----------------------------------------
| Seed: 3 | SUC | ...
-----------------------------------------
| Seed: 4 | SUC | ...
-----------------------------------------
| Seed: 5 | SUC | ...
-----------------------------------------
| Seed: 6 | SUC | ...
-----------------------------------------
| Seed: 7 | SUC | ...
-----------------------------------------
| Seed: 8 | SUC | ...
-----------------------------------------
| Seed: 9 | SUC | ...
-----------------------------------------
ACCURACY: 100.0%


    SUC: returns the target sequence fully reconstructed
    PAR: returns contigs all of which exist in the target sequence (consider coverage?)
    FAI: returns a full sequence that is incorrectly reconstructed or a set of contigs where at least one is not found in the target sequence

In [5]:
#! pip install Bio
from Bio import SeqIO

In [6]:
for record in SeqIO.parse("data/input/Raphanus sativus_NC_018551.1.fasta",'fasta'):
    sequence = record.seq
    # print(record)
    # break
reads = generate_reads(sequence,250,250,50,50,min_coverage=None,seed=0) 
G = build_bipartite_graph(reads) 

In [7]:
G_v,zero_in = build_virtual_graph(G,reads.copy())

In [8]:
path = sequitur(G,G_v,zero_in)

KeyboardInterrupt: 

In [ ]:
seq = find_path(path)

In [ ]:
seq == sequence

True

In [ ]:
seq

NameError: name 'seq' is not defined